In [1]:
import os
import dlib
import numpy as np
from skimage import io
import cv2

data_dir = os.path.expanduser('~/data')
faces_folder_path = data_dir + '/kodemaker/'

In [3]:
print(data_dir)
print(faces_folder_path)

/home/bidhya/data
/home/bidhya/data/kodemaker/


In [4]:
# Globals
face_detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor(data_dir + '/dlib/shape_predictor_68_face_landmarks.dat')
face_recognition_model = dlib.face_recognition_model_v1(data_dir + '/dlib/dlib_face_recognition_resnet_model_v1.dat')

RuntimeError: Unable to open /home/bidhya/data/dlib/shape_predictor_68_face_landmarks.dat

In [5]:
def get_face_encodings(face):
    bounds = face_detector(face, 1)
    faces_landmarks = [shape_predictor(face, face_bounds) for face_bounds in bounds]
    return [np.array(face_recognition_model.compute_face_descriptor(face, face_pose, 1)) for face_pose in faces_landmarks]

In [7]:
def get_face_matches(known_faces, face):
    return np.linalg.norm(known_faces - face, axis=1)

In [8]:
def find_match(known_faces, person_names, face):
    matches = get_face_matches(known_faces, face) # get a list of True/False
    min_index = matches.argmin()
    min_value = matches[min_index]
    if min_value < 0.55:
        return person_names[min_index]+"! ({0:.2f})".format(min_value)
    if min_value < 0.58:
        return person_names[min_index]+" ({0:.2f})".format(min_value)
    if min_value < 0.65:
        return person_names[min_index]+"?"+" ({0:.2f})".format(min_value)
    return 'Not Found'

In [9]:
def recognize_faces_in_video(face_encodings, person_names):
    faceClassifier = cv2.CascadeClassifier(data_dir + '/opencv/haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 360)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        face_rects = faceClassifier.detectMultiScale(
            gray,
            scaleFactor = 1.1,
            minNeighbors = 5,
            minSize = (50, 50),
            flags = cv2.CASCADE_SCALE_IMAGE)

        for (x, y, w, h) in face_rects:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            face = frame[y:y + h, x:x + w]
            face_encodings_in_image = get_face_encodings(face)
            if (face_encodings_in_image):
                match = find_match(face_encodings, person_names, face_encodings_in_image[0])
                cv2.putText(frame, match, (x+5, y-15), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

        cv2.imshow("bilde", frame)

        if cv2.waitKey(10) == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

In [10]:
face_encodings, person_names = load_face_encodings(faces_folder_path)
recognize_faces_in_video(face_encodings, person_names)

NameError: name 'load_face_encodings' is not defined